# Preparation

## Imports

In [ ]:
import os
import random
import json
import dotenv
from pymongo import MongoClient
from pymongo.server_api import ServerApi
from pymongo.errors import CollectionInvalid, DuplicateKeyError
from neo4j import GraphDatabase
from neo4j.exceptions import ClientError
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from faker import Faker

## Connect to MongoDB

In [ ]:
dotenv.load_dotenv()

mongo_user = os.getenv("MONGODB_USERNAME")
mongo_pass = os.getenv("MONGODB_PASSWORD")

mongo_client = MongoClient(
    (f"mongodb+srv://{os.getenv("MONGODB_USERNAME")}:{os.getenv("MONGODB_PASSWORD")}"
     "@projeto-bd.9scqvyv.mongodb.net/"
     "?retryWrites=true&w=majority&appName=projeto-bd"),
    server_api = ServerApi(
        version = "1",
        strict = True,
        deprecation_errors = True
    )
)

mongo_db = mongo_client["music_catalog"]

## Connect to Neo4j

In [ ]:
dotenv.load_dotenv()

neo4j = GraphDatabase.driver(
    "neo4j+s://10ab7e50.databases.neo4j.io",
    auth=(
        os.getenv("NEO4J_USERNAME"),
        os.getenv("NEO4J_PASSWORD"),
    ),
)

neo4j.verify_connectivity()

## Connect to Spotify API

In [ ]:
dotenv.load_dotenv()

spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials())

## Init Faker

In [ ]:
fake = Faker()

# Artists

## Create Entities

### MongoDB

In [ ]:
try:
    mongo_db.create_collection("artists")
    mongo_db.artists.create_index("releases.id")
except CollectionInvalid as e:
    print(e)

### Neo4j

In [ ]:
try:
    neo4j.execute_query("CREATE CONSTRAINT FOR (a:Artist) REQUIRE a.id IS UNIQUE")
    neo4j.execute_query("CREATE CONSTRAINT FOR (g:Genre) REQUIRE g.name IS UNIQUE")
    neo4j.execute_query("CREATE CONSTRAINT FOR (r:Release) REQUIRE r.id IS UNIQUE")
except ClientError as e:
    print(e.message)

## Retrieve Artists by Genre

### From Spotify

In [ ]:
response = spotify.search(
    q = "genre:djent",
    type = "artist",
)
artist_ids = [artist["id"] for artist in response["artists"]["items"]]

while response["artists"]["next"]:
    response = spotify.next(response["artists"])
    artist_ids.extend([artist["id"] for artist in response["artists"]["items"]])

artist_ids

### From JSON

In [ ]:
with open("../resources/my_guys.json", "r") as f:
    artist_ids = json.load(f)

artist_ids

## Insert into DBs

In [ ]:
def release_tracks(release_id: str) -> list:
    response = spotify.album_tracks(release_id)

    tracks = []
    for track in response["items"]:
        tracks.append({
            "track_number": track["track_number"],
            "name": track["name"],
            "duration": track["duration_ms"]
        })

    while response["next"]:
        response = spotify.next(response)
        for track in response["items"]:
            tracks.append({
                "track_number": track["track_number"],
                "name": track["name"],
                "duration": track["duration_ms"]
            })

    return tracks

def artist_releases(artist_id: str) -> list:
    response = spotify.artist_albums(artist_id, album_type="album")

    releases = []
    for release in response["items"]:
        releases.append({
            "id": release["id"],
            "name": release["name"],
            "release_date": release["release_date"],
            "tracks": release_tracks(release["id"]),
            "ratings": [],
        })

    while response["next"]:
        response = spotify.next(response)
        for release in response["items"]:
            releases.append({
                "id": release["id"],
                "name": release["name"],
                "release_date": release["release_date"],
                "tracks": release_tracks(release["id"]),
                "ratings": [],
            })

    return releases

for artist_id in artist_ids:
    response = spotify.artist(artist_id)

    artist = dict()
    artist["_id"] = artist_id
    artist["name"] = response["name"]
    artist["genres"] = response["genres"]
    artist["popularity"] = response["popularity"]
    if random.random() < 0.9:
        artist["bio"] = fake.paragraph(nb_sentences=25)
    artist["qt_followers"] = 0
    artist["releases"] = artist_releases(artist_id)

    if len(artist["releases"]) > 0:
        mongo_db.artists.insert_one(artist)

        neo4j.execute_query(
            """
            MERGE (a:Artist {id: $id})
            ON CREATE SET a.popularity = $popularity
            """,
            id = artist["_id"],
            popularity = artist["popularity"],
        )

        for genre in artist["genres"]:
            neo4j.execute_query(
                """
                MATCH (a:Artist {id: $artistId})
                MERGE (g:Genre {name: $name})
                MERGE (a)-[:BELONGS_TO]->(g)
                """,
                artistId = artist["_id"],
                name = genre,
            )

        for release in artist["releases"]:
            neo4j.execute_query(
                """
                MATCH (a:Artist {id: $artistId})
                MERGE (r:Release {id: $id})
                MERGE (a)-[:RELEASED]->(r)
                """,
                artistId = artist["_id"],
                id = release["id"],
            )

# Users

## Create Entities

### MongoDB

In [ ]:
try:
    mongo_db.create_collection("users")
    mongo_db.users.create_index("username", unique=True)
except CollectionInvalid as e:
    print(e)

### Neo4j

In [ ]:
try:
    neo4j.execute_query("CREATE CONSTRAINT FOR (u:User) REQUIRE u.username IS UNIQUE")
except ClientError as e:
    print(e.message)

## Insert into DBs

In [ ]:
QT_TRIES = 1_000
qt_fails = 0

for _ in range(QT_TRIES):
    try:
        first_name = fake.first_name()
        last_name = fake.last_name()
        number = random.randint(0, 99)

        user = dict()
        user["username"] = f"{first_name.lower()}_{last_name.lower()}{number:02}"
        user["password"] = fake.sha256()
        if random.random() < 0.75:
            user["name"] = f"{first_name} {last_name}"
        if "name" in user and random.random() < 0.5:
            user["bio"] = fake.paragraph(nb_sentences=10)
        user["friends"] = []
        user["artists_followed"] = []
        user["ratings"] = []

        mongo_db.users.insert_one(user)

        neo4j.execute_query(
            """
            MERGE (u:User {username: $username})
            """,
            username=user["username"],
        )
    except DuplicateKeyError:
        qt_fails += 1

print(f"Finished with {QT_TRIES - qt_fails} users inserted.")        

# Close Connections

In [ ]:
mongo_client.close()
neo4j.close()